<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)" data-toc-modified-id="Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description Task 1: RNN Language Modelling (30 +10 Points)</a></span><ul class="toc-item"><li><span><a href="#1a)-Language-Modelling-(30-Points)" data-toc-modified-id="1a)-Language-Modelling-(30-Points)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1a) Language Modelling (30 Points)</a></span></li><li><span><a href="#Conditional-Generation-(10-Points)" data-toc-modified-id="Conditional-Generation-(10-Points)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Conditional Generation (10 Points)</a></span></li></ul></li><li><span><a href="#Code-for-Task-1" data-toc-modified-id="Code-for-Task-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code for Task 1</a></span><ul class="toc-item"><li><span><a href="#Setup-and-preparation" data-toc-modified-id="Setup-and-preparation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup and preparation</a></span><ul class="toc-item"><li><span><a href="#Data-preprocessing" data-toc-modified-id="Data-preprocessing-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Data preprocessing</a></span></li></ul></li><li><span><a href="#RNN" data-toc-modified-id="RNN-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RNN</a></span></li><li><span><a href="#Experiment-A" data-toc-modified-id="Experiment-A-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Experiment A</a></span><ul class="toc-item"><li><span><a href="#Training-Experiment-A" data-toc-modified-id="Training-Experiment-A-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Training Experiment A</a></span></li><li><span><a href="#Testing-Experiment-A" data-toc-modified-id="Testing-Experiment-A-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Testing Experiment A</a></span></li></ul></li><li><span><a href="#Task-1.2:-Conditional-Generation-(Sampling)" data-toc-modified-id="Task-1.2:-Conditional-Generation-(Sampling)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Task 1.2: Conditional Generation (Sampling)</a></span></li></ul></li></ul></div>

# Natural Language Understanding: Project 1

[__Natural Language Understanding, Spring 2018, ETHZ__](http://www.da.inf.ethz.ch/teaching/2018/NLU/)

[__Project 1__ (ETHZ network)](http://www.da.inf.ethz.ch/teaching/2018/NLU/material/project.pdf)

# Project to-do list:

Somewhat in order of importance:

- ~~change code to unroll RNN in time instead of using dynamic_rnn~~
- ~~make sure the target data fed into the crossentropy metric is really in correct form~~
- ~~try own implementation of basic RNN cell instead of TF-prefab RNN or LSTM cell~~
- ~~change implementation to use the Xavier initializer instead of the uniform distribution currently used (see below)~~
- ~~change all `tf.Variable` variable inits to the better practice form like `W = tf.get_variable(name='example', shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())` which also includes the proper weight init~~
- ~~include dropout at input and/or RNN cell level for regularization~~
- ~~clean up namespaces, tensor naming~~
- ~~(Started 2.4.2018, but TBC)** build in all reporting for Tensorboard~~
- ~~adapt code to allow for differently sized timesteps~~
- ~~make arrangements to save trained model~~
- implement perplexity function
- ~~Maybe needs rewrite to use stock LSTM cell again** implement sampling function for conditional text generation~~
- ~~implement result output function~~
- ~~adapt code to allow for use of pretrained word2vec embedding~~

## Description Task 1: RNN Language Modelling (30 +10 Points)

### 1a) Language Modelling (30 Points)
Your task is to build a simple LSTM language model. To be precise, we assume that words are independent given the recurrent hidden state; we compute a new hidden state given the last hidden state and last word, and predict the next word given the hidden state:
$$ P(w_1,\dots,w_n) = 􏰀\prod_{t=1}^{n}P(w_t|\mathbf{h}_t)$$
$$ P(w_t|\mathbf{h}_t) = \text{softmax}(\mathbf{Wh}_t)$$
$$ \mathbf{h}_t = f(\mathbf{h}_{t−1}, w_{t-1}^{*})$$

where $f$ is the LSTM recurrent function, $\mathbf{W} \in \mathbb{R}^{|V|×d}$ are softmax weights and $\mathbf{h_0}$ is either an all-zero constant or a trainable parameter.
You can use the tensorflow cell implementation __[1](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell)__ to carry out the recurrent computation in $f$. However, you must construct the actual RNN yourself (e.g. don’t use tensorflow’s `static_rnn` or `dynamic_rnn` or any other RNN library). That means, you will need to use a python loop that sets up the unrolled graph. To make your life simpler, please follow these design choices:

__Model and Data specification__

- Use a special sentence-beginning symbol `<bos>` and a sentence-end symbol `<eos>` (please use exactly these, including brackets). The `<bos>` symbol is the input, when predicting the first word and the `<eos>` symbol you require your model to predict at the end of every sentence.
- Use a maximum sentence length of 30 (including the `<bos>` and `<eos>` symbol). Ignore longer sentences during training and testing.
- Use a special padding symbol `<pad>` (please use exactly this, including brackets) to fill up sentences of length shorter than 30. This way, all your input will have the same size.
- Use a vocabulary consisting of the 20K most frequent words in the training set, including the symbols `<bos>`, `<eos>`, `<pad>` and `<unk>`. Replace out-of-vocabulary words with the `<unk>` symbol before feeding them into your network (don’t change the file content).
- Provide the ground truth last word as input to the RNN, not the last word you predicted. This is common practice.
- Language models are usually trained to minimize the cross-entropy. Use tensorflow’s `tf.nn.sparse_softmax_cross_entropy_with_logits` to compute the loss (*This operation fuses the computation of the soft-max and the cross entropy loss given the logits. For numerical stability, it’s very important to use this function.*). Use the AdamOptimizer with default parameters to minimize the loss. Use `tf.clip_by_global_norm` to clip the norm of the gradients to 5.
- Use a batch size of 64.
- Use the data at __[6](https://polybox.ethz.ch/index.php/s/qUc2NvUh2eONfEB)__. Don’t pre-process the input further. All the data is already white-space tokenized and
lower-cased. One sentence per line.
- To initialize your weight matrices, use the `tf.contrib.layers.xavier_initializer()` initializer introduced in __[5](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf)__.

__Experiments__
All experiments should not run for longer than, say, four hours on the GPU. For this task, your
grade won’t improve with performance.

- __Experiment A__: Train your model with word-embedding dimensionality of 100 and a hidden state size of 512 and compute sentence perplexity on the evaluation set (see submission format below).
- __Experiment B__: It is common practice, to pretrain word embeddings using e.g. `word2vec`. This should make your model train faster as words will come already with some useful representation. Use the code at __[3](http://da.inf.ethz.ch/teaching/2018/NLU/material/load_embeddings.py)__ to load these word embeddings __[4](https://polybox.ethz.ch/index.php/s/cpicEJeC2G4tq9U)__ trained on the same corpus. Train your model again and compute evaluation perplexity.
- __Experiment C__: It is often desirable to make the LSTM more powerful, by increasing the hidden dimensionality. However, this will naturally increase the parameters $\mathbf{W}$ of the softmax. As a compromise, one can use a larger hidden state, but down-project it before the softmax. Increase the hidden state dimensionality from 512 to 1024, but down-project $h_t$ to dimensionality 512 before predicting $w_t$ as in
$$ \mathbf{\tilde{h}}_t = \mathbf{W}_P\mathbf{h}_t$$
where $W_P$ are parameters. Train your model again and compute evaluation perplexity.

__Submission and grading__
- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - __Three__ result files containing sentence-level perplexity numbers on the __test__ set (to be distributed) for
all three experiments. Recall that perplexity of a sentence $S = ⟨w_1, \dots , w_n⟩$ with respect to your model $p(w_t|w_1, \dots, w_{t−1})$ is defined as
$$ \text{Perp} = 2^{-\frac{1}{n} \sum_{t=1}^{n}\log_2 p(w_t|w_1,\dots,w_{t−1})}$$
The `<eos>` symbol is part of the sequence, while the `<pad>` symbols (if any) are not. Be sure to have the basis of the exponential and the logarithm match.<br>
__Input format sentences.test__<br>
One sentence (none of them is longer than 28 tokens) per line:<br>
         ```beside her , jamie bounced on his seat .
         i looked and saw claire montgomery looking up at me .
         people might not know who alex was , but they knew to listen to him .```<br>
__Required output format groupXX.perplexityY__<br>
(where XX is your group __number__ and Y ∈ {A,B,C} is the experiment). One perplexity number per line:<br>
         $10.232$<br>
         $2.434$<br>
         $5.232$<br>
Make sure to have equally many lines in the output as there are in the input – otherwise your submission will be rejected automatically.
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018

### Conditional Generation (10 Points)
Let’s use your trained language model from above to generate sentences. Given an initial sequence of words, your are asked to __greedily__ generate words until either your model decides to finish the sentence (it generated `<eos>`) or a given maximum length has been reached. Note, that this task does not involve any training. Please see the tensorflow documentation on how to save and restore your model from above.
There are several ways how to implement the generation. For example, you can define a graph that computes just one step of the RNN given the last input and the last state (both from a new placeholder).
$$ \text{state}_t, p_t = f(\text{state}_{t−1},w_{t−1}) $$
That means, for a prefix of size $m$ and a desired length of $n$, you run this graph $n$ times. The first $m + 1$ times you take the input form the prefix. For the rest of the sequence, you take the most likely2 word $w^{t−1} = \text{argmax}_w p_{t−1}(w)$ from the last step.

- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - Your continued sentences of length up to 20. Use your trained model from experiment __C__ in task 1.1.
    __Input format sentences.continuation__ One sentence (of length less than 20) per line:<br>
         ```beside her ,
         i
         people might not know```<br>
    The `<bos>` symbol is not explicitly in the file, but you should still use it as the first input.<br>
    __Required output format groupXX.continuation__ (where XX is your __group number__)<br>
         ```beside her , something happened ! <eos>
         i do n’t recall making a noise , but i must have , because bob just looked up from his
         people might not know the answer . <eos>```
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018


__Infrastructure__

You must use Tensorflow, but any programming language is allowed. However, we strongly recommend `python3`. You have access to two compute resources: Unlimited CPU usage on Euler and GPU usage on Leonhard. Note that the difference in speed is typically a factor between 10 and 100.

## Code for Task 1


### Setup and preparation

Make sure you have done the following:

- Download data from https://polybox.ethz.ch/index.php/s/qUc2NvUh2eONfEB and unpack into `./data/` subdirectory
- Download embeddings from https://polybox.ethz.ch/index.php/s/cpicEJeC2G4tq9U and unpack into `./data/` subdirectory
- Download helper function from http://da.inf.ethz.ch/teaching/2018/NLU/material/load_embeddings.py and put into `./helpers/` subdirectory

In [53]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # 0: ANY, 1: filter INFO, 2: filter WARNINGS, 3: filter ERROR
import time

import tensorflow as tf
import numpy as np

from collections import Counter
from gensim import models
from tqdm import tqdm

%matplotlib inline

In [51]:
# Read all data from files

data_dir = './data'

with open(data_dir+'/sentences.train', 'r') as f:
    train_data = f.read()
    
with open(data_dir+'/sentences.eval', 'r') as f:
    eval_data = f.read()
    
with open(data_dir+'/sentences.continuation', 'r') as f:
    continuation_data = f.read()    

In [11]:
# Set all config variables

class RNNConfig(object):
    '''Class holding all configuration vars for training the RNN
    
    '''
    
    def __init__(self,
                 train_data__path='./data/sentences.train',
                 eval_data__path='./data/sentences.eval',
                 validation_split=.1,
                 vocab_length=20000,
                 embedding_size=100,
                 rnn_size=512,
                 rnn_size_factor=1,
                 n_steps=10,
                 learning_rate=0.001,
                 keep_prob=0.5,
                 grad_clip=5.0,
                 batch_size=64,
                 num_epochs=10,
                 save_every_n=500,
                 validate_every_n=500,
                 report_every_n=100,
                 summary_every_n=10,
                 max_to_keep=5,
                 past_words=5):
        
        # Path to the training data 
        self.train_data__path = train_data__path
        
        # Path to the evaluation data 
        self.eval_data__path = eval_data__path
        
        # Percentage of the training data used for validation (default: 10%)
        self.validation_split = validation_split
        
        # Size of the vocabulary (default: 20k)
        self.vocab_length = vocab_length
        
        # Dimensionality of word embedding layer (default: 100)
        self.embedding_size = embedding_size
        
        # Dimensionality of RNN (i.e. hidden) layer (default: 512)
        self.rnn_size = rnn_size
        
        # Integer to factor the size of the hidden layer [Task 1.1C] (default: 1)
        self.rnn_size_factor = rnn_size_factor
        
        # Number of time steps for RNN (default: 30)
        self.n_steps = n_steps
        
        # Size of softmax output (default: vocab_length)
        self.out_size = self.vocab_length
        
        # Learning rate (default: 0.001)
        self.learning_rate = learning_rate
        
        # Dropout rate (default: 0.5)
        self.keep_prob = keep_prob
        
        # Gradient clipping treshold (default: 5.0)
        self.grad_clip = grad_clip
        
        # Batch Size (default: 64)
        self.batch_size = batch_size
        
        # Number of training epochs (default: 10)
        self.num_epochs = num_epochs
        
        # Save model after this many steps (default: 500)
        self.save_every_n = save_every_n
        
        # Validate after this many steps (default: 500)
        self.validate_every_n = validate_every_n
        
        # Print report after this many steps (default: 100)
        self.report_every_n = report_every_n
        
        # Record summary after this many steps (default: 10)
        self.summary_every_n = summary_every_n
        
        # Number of checkpoints to save (default: 5)
        self.max_to_keep = max_to_keep
        
        # How many previous words are used for prediction (default: 5)
        self.past_words = past_words

In [5]:
# Have a peek at the given raw data

print('Training data sample:\n', 20*'=')
print(train_data[:100], '\n', 80*'.')

print('\n Evaluation data sample:\n', 20*'=')
print(eval_data[:100], '\n', 80*'.')

print('\n Continuation data sample:\n', 20*'=')
print(continuation_data[:100], '\n', 80*'.')

Training data sample:
`` i 've never had any ice-cream for myself , my father never let me have any . ''
`` and even if sh 
 ................................................................................

 Evaluation data sample:
he took my face in his hands and held my face where he wanted it as he kissed me senseless .
`` on t 
 ................................................................................

 Continuation data sample:
`` no ,
correct ...
`` i
peter
what was i
when he
take some of
id told
throw
we have no place
i felt 
 ................................................................................


#### Data preprocessing

In [7]:
# Splitting data into sentences
def split_data2sentences(data):
    text = ''.join(data)
    sentences = text.split('\n')
    return sentences

train_sentences = split_data2sentences(train_data)
eval_sentences = split_data2sentences(eval_data)
continuation_sentences = split_data2sentences(continuation_data)

## We generate the dictionary from the training data, so this treatment is different

# Get sentences from training data and look at sample
print('Sample training sentences:\n', train_sentences[:5], '\n')

# Make text contiguous again, break into words for vocabulary and look at sample
words = ' '.join(train_sentences).split()
print('Sample words:\n', words[:20], '\n')

# Make a word counter and show top frequency words
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
print('Top frequency words:\n', vocab[:20], '\n')

# Clip word counter to defined length [20k] and append special symbol words
symbols = ['<bos>', '<eos>', '<pad>', '<unk>']
vocab = vocab[:CONFIG.vocab_length-len(symbols)] # Limit to CONFIG.vocab_length minus the last 4 to replace w/ symbols
for each in symbols:
    vocab.append(each)
    
# Make a vocabulary to convert words to integers
vocab_to_int = {word: i for i, word in enumerate(vocab, 0)} # consider starting with 1 if 0 gives dead cells

# Make a vocabulary to get words from integers at the end
int_to_vocab = dict(enumerate(vocab))


Sample training sentences:
 ["`` i 've never had any ice-cream for myself , my father never let me have any . ''", "`` and even if she was , the gargoyle could hardly halt me from harming her . ''", 'pigafetta repeated the threat , which had no apparent effect .', "best to barricade oneself , no ? ''", "`` why would he lie ? ''"] 

Sample words:
 ['``', 'i', "'ve", 'never', 'had', 'any', 'ice-cream', 'for', 'myself', ',', 'my', 'father', 'never', 'let', 'me', 'have', 'any', '.', "''", '``'] 

Top frequency words:
 ['.', ',', 'the', 'i', 'to', 'and', '``', "''", 'a', 'he', 'of', 'you', 'was', 'her', 'it', 'she', 'in', 'his', 'that', '?'] 



In [8]:
# Encode sentences to integers and insert symbol words where necessary

### ATTN: Remove next line after finishing, keeping data set small for speedup
train_sentences = train_sentences[:2000]

def encode_sentences(sentences):
    max_sentence_length = 30 # Given by task description
    sentences_ints = [] # List to hold converted-to-int sentences
    for each in tqdm(sentences):
        sentence = each.split()
        if len(sentence) < max_sentence_length-2: # -2 to allow for <bos>, <eos> \
                                                  # only use sentences w/ lenght <=30 as mandated
            sentence_int = [vocab_to_int['<bos>']] # Start sentence list w/ <bos>
            sentence_int += [vocab_to_int[word] if word in vocab_to_int\
                                 else vocab_to_int['<unk>'] for word in sentence] # Append remaining words
            sentence_int.append(vocab_to_int['<eos>']) # End sentence w/ <eos>
            while len(sentence_int) < max_sentence_length: # Pad length if necessary
                sentence_int.append(vocab_to_int['<pad>'])
            sentences_ints.append(sentence_int) 
    encoded = np.array(sentences_ints) # Convert list of sentences to np array
    return encoded

train_encoded = encode_sentences(train_sentences)
eval_encoded = encode_sentences(eval_sentences)
# Not needed with current procedure for continuation (see below):
# continuation_encoded = encode_sentences(continuation_sentences)

# Set data preparation complete flag
data_ready = True

100%|██████████| 10001/10001 [00:00<00:00, 64061.82it/s]


In [49]:
# Modified load_embedding code not to mess with own session handling

def load_embedding(vocab, path, dim_embedding, vocab_size):
    '''
      session        Tensorflow session object
      vocab          A dictionary mapping token strings to vocabulary IDs
      emb            Embedding tensor of shape vocabulary_size x dim_embedding
      path           Path to embedding file
      dim_embedding  Dimensionality of the external embedding.
      
      Returns np.ndarray of size (vocab_size, dim_embedding) containing pretrained embedding
    '''

    print("Loading external embeddings from %s" % path)

    model = models.KeyedVectors.load_word2vec_format(path, binary=False)  
    external_embedding = np.zeros(shape=(vocab_size, dim_embedding))
    matches = 0

    for tok, idx in vocab.items():
        if tok in model.vocab:
            external_embedding[idx] = model[tok]
            matches += 1
        else:
            print("%s not in embedding file" % tok)
            external_embedding[idx] = np.random.uniform(low=-0.25, high=0.25, size=dim_embedding)
        
    print("%d words out of %d could be loaded" % (matches, vocab_size))
    return external_embedding

In [54]:
# Load external embedding

filename = 'wordembeddings-dim100.word2vec'
external_embedding = load_embedding(vocab_to_int, data_dir+'/'+filename, CONFIG.embedding_size, CONFIG.vocab_length)

Loading external embeddings from ./data/wordembeddings-dim100.word2vec
<bos> not in embedding file
<eos> not in embedding file
<pad> not in embedding file
<unk> not in embedding file
19996 words out of 20000 could be loaded


### RNN 

In [9]:
def variable_summaries(var):
    '''Attach a lot of summaries to a Tensor (for TensorBoard visualization).
    
        From TensorBoard documentation
    '''
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [62]:
class RNNLanguageModel:
    '''Main element: Class representing the complete RNN language model
    
        TBC
    '''
    
    def __init__(self,
                 out_size,
                 batch_size,
                 n_steps,
                 rnn_size,
                 rnn_size_factor,
                 learning_rate,
                 grad_clip,
                 embedding_size,
                 sampling=False,
                 external_embedding=False):
        
        # Adapt batch_size, n_steps for sampling
        if (sampling == True):
             batch_size, n_steps = 1, 1
        
        # Reset tensorflow graph for clean slate
        tf.reset_default_graph()
        
        # Build input layer, keep_prob (all placeholders):
        with tf.name_scope('input_layer'):
            self.inputs = tf.placeholder(tf.int32, [batch_size, n_steps], name='inputs')
            self.targets = tf.placeholder(tf.int64, [batch_size, n_steps], name='targets')
            self.target_weights = tf.placeholder(tf.float32, [batch_size, n_steps],
                                                 name='target_weights')
            
            self.ext_embedding_matrix = tf.placeholder(tf.float32, [vocab_length, embedding_size],
                                                       name='ext_embedding_matrix')
            
            self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
            
            
            
        # Build embedding layer
        with tf.name_scope('embedding_layer'):
            # Create embedding matrix
            embedding_matrix = tf.get_variable(name='embedding_matrix',
                                           shape=[vocab_length, embedding_size],
                                           initializer=unif)

            # If we're using the pretrained embedding
            if (external_embedding == True):
                embedding_matrix.assign(self.ext_embedding_matrix)

            # Lookup inputs in embedding matrix
            self.embeddings = tf.nn.embedding_lookup(embedding_matrix, self.inputs,
                                                         name='embeddings')
            
            tf.summary.histogram('embeddings', self.embeddings)
            tf.summary.scalar('dropout', keep_prob)
            
            # Embedding layer dropout (i.e. during training)
            if (keep_prob < 1):
                self.embeddings = tf.nn.dropout(self.embeddings, keep_prob, name='embeddings_dropout')
        
        # Build RNN layer
        with tf.name_scope('hidden_layer'):
            
            # LSTM cell with dropout wrapper
            cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size*rnn_size_factor)
            cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=keep_prob)
            
            # Initialize cell to zero state
            self.initial_state = cell.zero_state(batch_size, tf.float32)
    
            # Unroll RNN through time
            # dynamic_rnn solution would be:
            #  self.rnn_output, self.final_state = tf.nn.dynamic_rnn(cell, self.embeddings, initial_state=self.initial_state)

            state = self.initial_state 
            outputs = []
            for i in range(n_steps):
                output, state = cell(self.embeddings[:, i, :], state)
                outputs.append(output)
            # to try out: feed 'ground truth' as last input: needs lookup of targets
            self.rnn_output, self.final_state = tf.stack(outputs, axis=1), state
                        
            # to keep only last output would be:
            # self.rnn_output = self.rnn_output[:, -1, :]
        
            # Reshape hidden layer output: one row per input and step, i.e. ((batch_size*n_steps), rnn_size)            
            self.rnn_output = tf.reshape(self.rnn_output, [-1, rnn_size*rnn_size_factor])
                        
            # If hidden layer increased by factor [1.1C], project down
            if (rnn_size_factor > 1):
                W_p = tf.get_variable(name='downprojection_weight', shape=[rnn_size*rnn_size_factor,
                                                                          rnn_size],
                                      initializer=xavi)
                b_p = tf.get_variable(name='downprojection_bias', shape=[rnn_size,],
                                      initializer=zeros)
                
                self.rnn_output = tf.nn.xw_plus_b(self.rnn_output, W_p, b_p)
                
                variable_summaries(W_p)
                variable_summaries(b_p)
                
        # Softmax output layer
        with tf.name_scope('softmax_layer'):
            # Hook up RNN outputs to softmax layer:
            W_softmax = tf.get_variable(name="softmax_weight", shape=[rnn_size, out_size],
                                        initializer=xavi)
            b_softmax = tf.get_variable(name="softmax_bias", shape=[out_size],
                                        initializer=zeros)
            
            variable_summaries(W_softmax)
            variable_summaries(b_softmax)
            
            # Calculate logits from softmax layer
            self.logits = tf.nn.xw_plus_b(self.rnn_output, W_softmax, b_softmax, name='logits')
            
            tf.summary.histogram('logits', self.logits)
            
            # Finally, get word probabilities from logits
            self.predictions = tf.nn.softmax(self.logits, name='predictions')
            
            tf.summary.histogram('predictions', self.predictions)
        
        # Loss and accuracy
        with tf.name_scope('loss_accuracy'):

            # Reshape targets from (batch_size, n_steps) to 1D
            y = tf.reshape(self.targets, [-1])
            
            # Calculate crossentropy: use sparse routine and not
            # softmax_cross_entropy_with_logits from task description as it is deprecated
            crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                      logits=self.logits)

            # Multiply crossentropy vector by 1D-reshaped weights to nullify effect of <pad>
            crossent *= tf.reshape(self.target_weights, [-1])
            
            # Get scalar crossentropy
            crossent = tf.reduce_sum(crossent)

            # Calculate total weight and make sure it doesn't equal zero for the division below
            total_weight = tf.reduce_sum(self.target_weights)
            total_weight += 1e-12

            # Calculate loss: Divide crossentropy (across batch_size AND n_steps) through total weight
            self.loss = crossent / total_weight
            
            tf.summary.scalar('loss', self.loss)

            # Best prediction
            self.best_prediction = tf.argmax(self.predictions, 1, name='best_prediction')
            
            # Accuracy
            correct_prediction = tf.equal(self.best_prediction, y)
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
            tf.summary.scalar('accuracy', self.accuracy)

        # Optimizer
        with tf.name_scope('optimizer'):
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), grad_clip)
            train_op = tf.train.AdamOptimizer(learning_rate)
            self.optimizer = train_op.apply_gradients(zip(grads, tvars))      

        # Merge summaries for TensorBoard    
        self.merged = tf.summary.merge_all()

In [30]:
# Build batch generator for training    
def get_batches(source_arr, batch_size, n_steps):
    '''Generator which returns features x and targets y
    
    Inputs
    ------
    source_arr: A np.ndarray of sentences in rows to generate features and targets from
    batch_size: An int number of sequences required per batch
    n_steps: Number of time steps for RNN to consider; defines sequence length
    
    Outputs
    -------
    x: A np.ndarray of feature sequences according to parameters above
    y: A np.ndarray of target sequences according to parameters above
    y_weights: A np.ndarray of floats: 0.0 were target sequence element = <pad>, 1.0 else
    '''
    
    source_rows = source_arr.shape[0]
    source_cols = source_arr.shape[1]
    
    # Make sure time steps doesn't exceed available information in sentence
    assert (n_steps <  source_cols), "Can't look further back than sentence length"
    
    # How many sequences we can get from an input sentence (row)
    n_seq_per_row = source_cols - n_steps 
        
    # How many 'batch blocks' we can get
    n_blocks = source_rows // batch_size
   
    # Shuffle rows of sequences to improve training
    np.random.shuffle(source_arr)
    
    # Crop array to only produce full batches
    source_arr = source_arr[:n_blocks*batch_size, :]

    # Reshape source_arr for easier batch generation
    source_arr = source_arr.reshape((batch_size, -1))
    
    # Generate batches
    for j in range(0, source_arr.shape[1], source_cols):
        for jj in range(n_seq_per_row):
            # Feature sequence:
            x = source_arr[:, j+jj  :j+jj+n_steps  ]
            # Target:
            y = source_arr[:, j+jj+1:j+jj+n_steps+1]
            # Target weights:
            y_weights = np.array([[0. if (int_to_vocab[j] == '<pad>') else 1. for j in i] for i in y])
            yield x, y, y_weights

In [37]:
# Development only: playing around with batches

shuffled_rows_ind = np.random.permutation(len(train_encoded))
validation_split_ind = int(0.1 * len(train_encoded))
source_train = train_encoded[shuffled_rows_ind[validation_split_ind:]]
batches = get_batches(source_train, 3, 8)

x, y, y_weights = next(batches)
print('x\n',x.shape,'\n', x, '\n')
print('y\n',y.shape,'\n', y, '\n')
print('y_weights\n',y_weights.shape,'\n', y_weights, '\n')
for i in range(x.shape[0]):
        print([int_to_vocab[x[i,j]] for j in range(x[i].shape[0])], ' -> ', [int_to_vocab[y[i,j]] for j in range(y[i].shape[0])])

x
 (3, 8) 
 [[19996    30  1941    12   111    35    71  1632]
 [19996     2  3074   160    49     4 12379    23]
 [19996     6  3576    67 19997 19998 19998 19998]] 

y
 (3, 8) 
 [[   30  1941    12   111    35    71  1632     9]
 [    2  3074   160    49     4 12379    23     1]
 [    6  3576    67 19997 19998 19998 19998 19998]] 

y_weights
 (3, 8) 
 [[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0.]] 

['<bos>', 'but', 'guilt', 'was', 'still', 'not', 'an', 'emotion']  ->  ['but', 'guilt', 'was', 'still', 'not', 'an', 'emotion', 'he']
['<bos>', 'the', 'echo', 'came', 'back', 'to', 'taunt', 'me']  ->  ['the', 'echo', 'came', 'back', 'to', 'taunt', 'me', ',']
['<bos>', '``', 'ha', '!', '<eos>', '<pad>', '<pad>', '<pad>']  ->  ['``', 'ha', '!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']


### Experiment A
#### Training Experiment A

In [66]:
### A: Run training

experiment = '-exp-A'

# Define shorthands for common initializers
ones = tf.ones_initializer
unif = tf.random_uniform_initializer(-1, 1)
xavi = tf.contrib.layers.xavier_initializer()
zeros = tf.zeros_initializer()

# Set configuration for training
CONFIG = RNNConfig(num_epochs=1, report_every_n=20, validate_every_n=250,
                   rnn_size_factor=1, n_steps=5)

# Check data preparation complete
assert data_ready == True, 'Need to run data preparation first'

# Split training/validation data
np.random.seed(42)
shuffled_rows_ind = np.random.permutation(len(train_encoded))
validation_split_ind = int(CONFIG.validation_split * len(train_encoded))
source_train = train_encoded[shuffled_rows_ind[validation_split_ind:]]
source_validation = train_encoded[shuffled_rows_ind[:validation_split_ind]]

# Set training loop variables
num_epochs = CONFIG.num_epochs
save_every_n = CONFIG.save_every_n
validate_every_n = CONFIG.validate_every_n
report_every_n = CONFIG.report_every_n
summary_every_n = CONFIG.summary_every_n
keep_prob = CONFIG.keep_prob
report_detail = False
if CONFIG.n_steps < 15:
    report_detail = True

# Create model instance
model = RNNLanguageModel(out_size=CONFIG.out_size,
                         batch_size=CONFIG.batch_size,
                         n_steps=CONFIG.n_steps,
                         rnn_size=CONFIG.rnn_size,
                         rnn_size_factor=1, # Experiment A
                         learning_rate=CONFIG.learning_rate,
                         grad_clip=CONFIG.grad_clip,
                         embedding_size=CONFIG.embedding_size,
                         external_embedding=False) # Experiment A

# Setup TensorBoard logging
now = time.strftime('%y-%m-%d-%H-%M-%S')
log_dir = './runs'+experiment
log_subdir = '{}/run-{}/'.format(log_dir, now)
writer = tf.summary.FileWriter(log_subdir, tf.get_default_graph())

# Setup model saving
save_dir = './checkpoints'+experiment
saver = tf.train.Saver(max_to_keep=CONFIG.max_to_keep)

# Run training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    
    for ep in range(num_epochs):
        new_state = sess.run(model.initial_state)
        loss = 0
        
        validation_batches = get_batches(source_validation, batch_size=CONFIG.batch_size,
                                n_steps=CONFIG.n_steps)
        
        for x_train, y_train, y_train_weights in get_batches(source_train, batch_size=CONFIG.batch_size,
                                n_steps=CONFIG.n_steps):
            counter += 1
            start = time.time()
            
            feed_dict = {model.inputs: x_train,
                         model.targets: y_train,
                         model.target_weights: y_train_weights,
                         model.keep_prob: keep_prob,
                         #model.ext_embedding_matrix: external_embedding, # Experiment A
                         model.initial_state: new_state}
            
            loss, accuracy, best_prediction, new_state, _ = \
            sess.run([model.loss, model.accuracy, model.best_prediction, model.final_state,
                      model.optimizer], feed_dict=feed_dict)
            
            batch_time = time.time() - start
            
            if (counter % summary_every_n == 0):
                writer.add_summary(sess.run(model.merged, feed_dict=feed_dict), counter)
            
            if (counter % report_every_n == 0):
                print('Epoch: {}/{}\t'.format(ep+1, num_epochs),
                      'Training step: {}\t'.format(counter),
                      'Batch loss: {:.3f}\t'.format(loss),
                      'Batch accuracy: {:.1%}\t'.format(accuracy),
                      '{:.2f} sec/batch'.format(batch_time))
                if (report_detail == True):
                    print(' '.join([int_to_vocab[x_train[0,j]] for j in range(x_train[0].shape[0])]),
                          ' -> ', int_to_vocab[best_prediction[0]] , ' vs. ', int_to_vocab[y_train[0, -1]])
            
            if (counter % save_every_n == 0):
                saver.save(sess, '{}/run-{}_i{}_s{}.ckpt'.format(save_dir, now,
                                                                 counter, CONFIG.rnn_size))
            if (counter % validate_every_n == 0):
                x_validation, y_validation, y_validation_weights = next(validation_batches)
                feed_dict = {model.inputs: x_validation,
                             model.targets: y_validation,
                             model.target_weights: y_validation_weights,
                             model.keep_prob: 1.0,
                             #model.ext_embedding_matrix: external_embedding, # Experiment A
                             model.initial_state: new_state}
                
                loss, accuracy = sess.run([model.loss, model.accuracy], feed_dict=feed_dict)
                print('Epoch: {}/{}\t'.format(ep+1, num_epochs),
                      '*** Validation ***\t',
                      'Loss: {:.3f}'.format(loss),
                      'Accuracy: {:.1%}\t'.format(accuracy))
                
    writer.close()
    saver.save(sess, '{}/run-{}_i{}_s{}.ckpt'.format(save_dir, now, counter, CONFIG.rnn_size))

Epoch: 1/1	 Training step: 20	 Batch loss: 4.405	 Batch accuracy: 2.2%	 0.39 sec/batch
<pad> <pad> <pad> <pad> <pad>  ->  <eos>  vs.  <pad>


KeyboardInterrupt: 

#### Testing Experiment A

In [71]:
def eval_perplexities(checkpoint):
    perplexities = []
    saver = tf.train.Saver()
    for s in eval_encoded:
        test_feed = []
        [test_feed.append(w) if not (int_to_vocab[w] == '<pad>') else None for w in s]
        model = RNNLanguageModel(out_size=CONFIG.out_size,
                     batch_size=1,
                     n_steps=len(test_feed),
                     rnn_size=CONFIG.rnn_size,
                     rnn_size_factor=CONFIG.rnn_size_factor,
                     learning_rate=CONFIG.learning_rate,
                     grad_clip=CONFIG.grad_clip,
                     embedding_size=CONFIG.embedding_size)

        
        for x_train, y_train, y_train_weights in get_batches(source_train, batch_size=CONFIG.batch_size,
                                n_steps=CONFIG.n_steps):
        
        
        with tf.Session() as sess:
            saver.restore(sess, checkpoint)
            new_state = sess.run(model.initial_state)
            x = np.atleast_2d(test_feed)
            feed_dict = {model.inputs: x,
                         model.keep_prob: 1.0,
                         model.initial_state: new_state}
            perplexity = sess.run(model.perplexity, feed_dict=feed_dict)
            print
            
            [print(int_to_vocab[i]) for i in test_feed]
    return perplexities

In [73]:
checkpoint = tf.train.latest_checkpoint(save_dir)
eval_perplexities(checkpoint)

<bos>
he
took
my
face
in
his
hands
and
held
my
face
where
he
wanted
it
as
he
kissed
me
senseless
.
<eos>
<bos>
``
on
the
contrary
,
your
majesty
,
''
said
the
dwarf
.
<eos>
<bos>
i
hope
she
does
n't
care
.
<eos>
<bos>
when
i
get
my
chance
,
im
gone
and
i
advise
you
to
do
the
same
likewise
.
<eos>
<bos>
i
did
as
he
told
me
.
<eos>
<bos>
``
why
do
n't
you
come
up
to
my
room
so
i
can
change
.
''
<eos>
<bos>
i
'll
tell
you
the
problem
with
engineers
and
scientists
.
<eos>
<bos>
so
that
when
we
go
meet
your
family
,
there
is
no
doubt
to
anyone
looking
at
us
that
you
're
happy
and
confident
.
''
<eos>
<bos>
sydney
couldnt
move
.
<eos>
<bos>
has
carl
stopped
by
yet
?
''
<eos>
<bos>
<unk>
.
<eos>
<bos>
jamie
stepped
up
beside
her
and
took
her
firmly
by
the
elbow
.
<eos>
<bos>
none
of
them
took
notice
of
me
as
they
bustled
about
their
day
.
<eos>
<bos>
i
repeated
.
<eos>
<bos>
they
entered
.
<eos>
<bos>
the
two
on
the
corners
looked
<unk>
with
scars
to
prove
it
.
<eos>
<bos>
we
find
it
mildly
a

?
<eos>
<bos>
i
walked
over
as
he
was
getting
the
bolt
<unk>
in
place
around
the
lock
pin
.
<eos>
<bos>
``
i
...
<eos>
<bos>
he
's
coming
over
tomorrow
to
explain
to
me
why
he
left
.
<eos>
<bos>
ty
insisted
,
sounding
a
mixture
of
exasperated
and
hurt
that
zane
did
n't
believe
him
.
<eos>
<bos>
``
we
were
taught
to
respect
women
,
''
will
adds
with
a
husky
voice
.
<eos>
<bos>
thus
strength
of
union
was
borne
.
<eos>
<bos>
my
hackles
rose
,
and
i
could
n't
shake
the
nagging
feeling
of
betrayal
.
<eos>
<bos>
``
no
one
took
any
pleasure
in
the
death
of
torak
.
<eos>
<bos>
<unk>
went
to
the
closet
,
flipped
<unk>
until
she
found
a
sunny
yellow
dress
,
then
tossed
it
onto
the
bed
.
<eos>
<bos>
bridger
clamped
his
hand
tightly
around
his
throat
.
<eos>
<bos>
drunk
driving
was
bad
enough
,
but
with
an
injury
to
boot
,
well
,
it
would
have
destroyed
him
.
<eos>
<bos>
smiling
again
,
he
went
to
get
a
drink
.
<eos>
<bos>
``
do
n't
you
love
him
at
all
,
reah
?
''
<eos>
<bos>
he
was
breathing
hard

my
eyes
fluttered
open
when
the
lap
my
head
was
resting
in
moved
.
<eos>
<bos>
he
put
that
out
of
his
mind
by
trying
to
ignore
the
<unk>
of
his
stomach
.
<eos>
<bos>
maybe
that
's
his
problem
.
''
<eos>
<bos>
around
him
,
the
station
walls
formed
angles
at
the
center
to
make
an
elongated
<unk>
.
<eos>
<bos>
im
not
sure
,
honey
.
<eos>
<bos>
ms.
terwilliger
had
told
me
the
invisibility
spell
would
protect
me
from
video
and
film
.
<eos>
<bos>
like
,
vomit
blood
sick
.
<eos>
<bos>
when
finally
i
climbed
my
way
up
to
her
,
i
placed
my
hands
on
either
side
of
her
head
.
<eos>
<bos>
i
just
could
n't
manage
to
keep
my
distance
from
you
.
''
<eos>
<bos>
ava
shifted
uncomfortably
in
her
seat
,
caught
herself
,
and
stilled
.
<eos>
<bos>
i
think
she
's
gone
to
find
someone
.
<eos>
<bos>
``
no
way
,
''
kendra
said
.
<eos>
<bos>
his
jaw
worked
.
<eos>
<bos>
i
can
only
know
that
i
'm
going
to
fight
whatever
happens
to
make
sure
you
're
safe
.
''
<eos>
<bos>
she
did
not
intend
to
ignore
the
warnings


his
gaze
went
to
the
windows
,
but
he
could
not
see
cassie
amongst
the
crush
of
students
that
had
finished
eating
and
moved
outside
.
<eos>
<bos>
burnett
's
body
language
and
emotions
said
what
he
thought
of
that
idea
.
<eos>
<bos>
``
and
remember
,
i
am
always
watching
.
''
<eos>
<bos>
let
's
get
your
things
and
we
'll
go
to
my
house
and
hang
out
.
''
<eos>
<bos>
every
.
<eos>
<bos>
``
probably
,
yes
.
<eos>
<bos>
the
dorm
looked
majestic
to
her
.
<eos>
<bos>
``
that
's
right
,
''
the
woman
agreed
.
<eos>
<bos>
a
translucent
new
shape
<unk>
adam
's
body
,
shining
here
and
there
with
deep
,
fiery
orange
.
<eos>
<bos>
nick
cackled
and
shook
his
head
.
<eos>
<bos>
not
since
i
'd
told
him
that
i
was
pregnant
,
actually
.
<eos>
<bos>
wanted
to
tell
you
,
it
wasnt
planned
,
she
said
.
<eos>
<bos>
allie
asked
.
<eos>
<bos>
also
obvious
was
how
much
silas
stood
out
,
his
being
the
only
person
sitting
in
one
of
the
more
than
three
hundred
seats
available
.
<eos>
<bos>
her
gaze
cut
to
dylan
.
<

cassie
had
told
him
that
was
often
where
they
held
their
training
practices
.
<eos>
<bos>
sera
was
used
to
noise
,
activity
and
chaos
.
<eos>
<bos>
he
made
not
a
sound
as
she
carried
him
to
her
bed
,
but
she
was
somewhat
surprised
at
the
solid
weight
of
him
.
<eos>
<bos>
its
boring
but
ill
get
a
decent
job
.
<eos>
<bos>
``
you
are
deeply
loved
.
''
<eos>
<bos>
one
could
presumably
travel
quickly
between
distant
places
by
,
say
,
entering
the
silent
city
in
new
york
city
and
exiting
in
tokyo
.
<eos>
<bos>
no
matter
what
she
felt
or
what
she
had
felt
last
august
...
that
part
of
her
life
was
over
.
<eos>
<bos>
sara
?
<eos>
<bos>
``
good
way
of
looking
at
it
.
<eos>
<bos>
the
mouth
widened
<unk>
as
the
lips
drew
back
and
drew
back
,
meeting
the
nose
and
disappearing
in
an
oral
ring
of
jutting
teeth
.
<eos>
<bos>
i
smiled
,
my
head
still
resting
against
his
chest
,
almost
above
the
tattoo
of
my
name
.
<eos>
<bos>
it
was
gloomy
under
the
trees
,
and
i
wished
i
could
walk
closer
to
edward
an

...

,
smiling
.
<eos>
<bos>
``
i
do
n't
know
,
reyes
.
<eos>
<bos>
you
've
had
too
much
today
,
man
.
''
<eos>
<bos>
move
along
,
kate
.
<eos>
<bos>
it
's
so
nice
out
here
.
''
<eos>
<bos>
not
good
old
danny
boy
.
<eos>
<bos>
``
i
still
hate
flying
.
<eos>
<bos>
it
was
big
business
for
the
local
<unk>
<unk>
.
<eos>
<bos>
jaikus
rolled
his
eyes
as
<unk>
took
the
lead
with
<unk>
right
behind
pointing
the
way
.
<eos>
<bos>
harmony
,
listen
,
if
youre
worried
about
me
pressing
charges
,
theres
no
need
.
<eos>
<bos>
the
lens
is
busted
,
the
film
is
shredded
,
and
the
camera
is
crushed
.
<eos>
<bos>
justin
vaguely
remembered
a
night
out
with
her
.
<eos>
<bos>
she
looked
like
my
<unk>
again
.
<eos>
<bos>
``
time
for
the
results
!
''
<eos>
<bos>
im
glad
that
we
decided
to
return
early
from
our
luna
de
<unk>
(
honeymoon
)
.
<eos>
<bos>
the
platoon
boarded
,
and
on
the
way
up
,
the
<unk>
swooped
by
to
pick
up
harlin
and
<unk>
.
<eos>
<bos>
<unk>
did
n't
always
look
like
a
<unk>
with
a
<unk>
.
<eos>


so
he
could
cough
up
the
water
.
<eos>
<bos>
asked
dan
.
<eos>
<bos>
the
<unk>
were
close
by
.
<eos>
<bos>
``
charlotte
gave
me
one
of
his
books
.
<eos>
<bos>
i
slip
an
arm
around
her
waist
.
<eos>
<bos>
now
.
''
<eos>
<bos>
i
felt
completely
empty
.
<eos>
<bos>
i
held
her
arms
.
<eos>
<bos>
the
woman
,
who
believes
she
spawned
me
,
slices
the
air
with
her
hostile
tone
.
<eos>
<bos>
if
he
could
have
looked
into
his
own
dark
soul
,
he
would
have
seen
a
dim
light
.
<eos>
<bos>
<unk>
jason
heard
'em
preaching
once
down
in
<unk>
and
they
was
saying
some
very
nasty
things
about
witches
,
'
said
nanny
.
<eos>
<bos>
``
i
mean
it
,
''
miranda
says
.
<eos>
<bos>
you
cant
charge
me
with
anything
.
<eos>
<bos>
``
not
if
i
call
.
<eos>
<bos>
if
she
were
giving
a
friend
advice
in
this
situation
,
what
would
she
say
?
<eos>
<bos>
mary
forced
a
smile
.
<eos>
<bos>
encircled
and
with
nowhere
to
run
,
he
dropped
to
his
knees
in
defeat
.
<eos>
<bos>
the
girls
with
me
.
<eos>
<bos>
nicole
sauntered
up
be

complicated
series
of
chords
and
the
fans
behind
her
shouted
their
approval
.
<eos>
<bos>
``
not
very
good
actually
.
<eos>
<bos>
we
're
traditional
that
way
.
''
<eos>
<bos>
he
waved
his
arms
dramatically
.
<eos>
<bos>
rebecca
would
still
be
packing
,
but
perhaps
he
could
entice
her
to
stop
long
enough
for
an
early
luncheon
.
<eos>
<bos>
i
thought
back
to
the
memory
.
<eos>
<bos>
``
that
's
a
<unk>
,
''
sullivan
said
.
<eos>
<bos>
the
bathroom
is
fitted
up
with
toiletries
.
<eos>
<bos>
are
we
all
leaving
?
<eos>
<bos>
he
moved
his
hand
to
where
his
gun
normally
was
.
<eos>
<bos>
they
were
soft
and
young
and
somehow
<unk>
,
and
made
their
deaths
that
much
more
offensive
.
<eos>
<bos>
she
looked
like
she
was
wasted
.
<eos>
<bos>
there
was
no
way
she
'd
open
herself
up
to
him
.
<eos>
<bos>
``
what
's
the
book
called
,
dad
?
''
<eos>
<bos>
regret
filled
him
,
and
when
he
looked
at
the
moon
's
face
,
it
seemed
mournful
to
him
.
<eos>
<bos>
then
i
reluctantly
<unk>
his
fingers
from
mine
.
<

<bos>
and
i
do
n't
have
jurisdiction
.
''
<eos>
<bos>
``
i
'm
...
we
're
...
are
you
...
?
''
<eos>
<bos>
then
he
began
tightening
the
straps
.
<eos>
<bos>
she
pauses
.
<eos>
<bos>
ernie
slowed
them
down
and
stopped
on
the
first
page
for
the
paper
dated
july
19
,
<unk>
.
<eos>
<bos>
which
,
was
a
feat
unto
itself
because
i
had
n't
been
sure
i
was
capable
of
such
<unk>
.
<eos>
<bos>
his
mouth
forms
a
grim
line
,
and
he
's
silent
for
a
moment
.
<eos>
<bos>
she
could
find
out
about
that
taylor
pork
roll
and
cheese
with
egg
on
a
<unk>
you
had
last
week
.
<eos>
<bos>
robert
said
the
film
director
wants
me
to
sing
a
spanish
song
in
one
of
the
scenes
.
<eos>
<bos>
now
this
evening
visitor
,
if
it
was
her
visitor
,
was
a
<unk>
.
<eos>
<bos>
``
and
you
still
have
the
elves
to
<unk>
,
''
gabriel
said
.
<eos>
<bos>
she
did
n't
have
to
.
<eos>
<bos>
there
's
a
beat
of
silence
on
the
other
end
,
and
i
start
to
panic
.
<eos>
<bos>
as
he
did
so
,
his
<unk>
went
off
again
.
<eos>
<bos>
it
was
n't
the


<eos>
<bos>
``
there
is
n't
any
reason
you
<unk>
''
``
we
ca
n't
heal
,
''
he
interrupted
,
``
unless
we
try
.
<eos>
<bos>
``
my
evening
star
,
you
are
full
of
surprises
.
''
<eos>
<bos>
``
come
on
.
<eos>
<bos>
``
speaking
of
going
back
,
i
think
we
're
good
enough
friends
now
for
you
to
tell
me
who
your
first
kiss
was
.
''
<eos>
<bos>
``
<unk>
seven
?
''
<eos>
<bos>
when
,
at
precisely
nine
o'clock
,
isabella
did
open
the
door
without
knocking
,
mac
was
bent
over
his
<unk>
mixing
paints
.
<eos>
<bos>
``
so
it
's
twenty
years
minimum
?
''
<eos>
<bos>
``
come
on
in
.
''
<eos>
<bos>
then
he
stood
,
giving
ian
a
few
necessary
seconds
to
recover
from
the
<unk>
of
the
kid
's
gratitude
.
<eos>
<bos>
i
was
n't
so
sure
i
could
do
it
again
,
especially
if
everyone
was
kind
of
counting
on
it
.
<eos>
<bos>
i
released
his
waist
and
stepped
back
so
i
could
meet
his
eyes
full-on
.
<eos>
<bos>
'it
does
help
me
.
<eos>
<bos>
here
is
the
girl
i
interviewed
at
a
cool
little
bar
called
la
esmeralda
:
ca

In [ ]:
# Get latest checkpoint
checkpoint = tf.train.latest_checkpoint(save_dir)

# Calculate perplexities:
perplexities = eval_perplexities(checkpoint)

# Write to file
out_dir = './outputs'
filename = 'group01.perplexityA'
if not os.path.exists(out_dir):
        os.makedirs(out_dir)
with open('{}/{}'.format(out_dir, filename), 'w') as f:
    for p in perplexities:
        f.write('{}\n'.format(p))

### Task 1.2: Conditional Generation (Sampling)

In [74]:
def generate_samples(checkpoint, n_samples, primer):
    samples = [w for w in primer.split()]
    primer_feed = [vocab_to_int['<bos>']]
    [primer_feed.append(vocab_to_int[word]) for word in samples]
    model = RNNLanguageModel(out_size=CONFIG.out_size,
                             batch_size=CONFIG.batch_size,
                             n_steps=CONFIG.n_steps,
                             rnn_size=CONFIG.rnn_size,
                             rnn_size_factor=CONFIG.rnn_size_factor,
                             learning_rate=CONFIG.learning_rate,
                             grad_clip=CONFIG.grad_clip,
                             embedding_size=CONFIG.embedding_size,
                             sampling=True)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for i in primer_feed:
            x = np.zeros((1,1)) # try: x = np.atleast_1d(i)
            x[0, 0] = i
            feed_dict = {model.inputs: x,
                         model.keep_prob: 1.0,
                         model.initial_state: new_state}
            best_prediction, new_state = \
            sess.run([model.best_prediction, model.final_state], feed_dict=feed_dict)
            
        samples.append(int_to_vocab[best_prediction[0]])
        for j in range(n_samples):
            x[0, 0] = best_prediction
            feed_dict = {model.inputs: x,
                         model.keep_prob: 1.0,
                         model.initial_state: new_state}
            best_prediction, new_state = \
            sess.run([model.best_prediction, model.final_state], feed_dict=feed_dict)
            samples.append(int_to_vocab[best_prediction[0]])
            if (samples[-1] == '<eos>'):
                break
    return ' '.join(samples)

In [51]:
# Get latest checkpoint
checkpoint = tf.train.latest_checkpoint(save_dir)
# checkpoint = './checkpoints/run-18-04-07-18-21-43_i19000_s512.ckpt'

# Sentence generation length limit
max_generate_n = 20

# Generate continuation text
n_samples = 20 # Dev only, use next line for production
#n_samples = len(continuation_sentences)
continued_sentences = []
for i in range(n_samples):
    sample = generate_samples(checkpoint, max_generate_n, continuation_sentences[i])
    continued_sentences.append(sample)

# Have a look
for s in np.random.permutation(n_samples)[:5]:
    print(continuation_sentences[s], ' -> ', continued_sentences[s])
    
# Write to file
out_dir = './outputs'
filename = 'group01.continuation'
if not os.path.exists(out_dir):
        os.makedirs(out_dir)
with open('{}/{}'.format(out_dir, filename), 'w') as f:
    for s in continued_sentences:
        f.write('{}\n'.format(s))

INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-14-21-18_i675_s512.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoints/run-18-04-08-1

In [67]:
eval_encoded[0]

array([19996,     9,   146,    20,   113,    16,    17,   154,     5,
         275,    20,   113,   127,     9,   147,    14,    28,     9,
         602,    23, 10167,     0, 19997, 19998, 19998, 19998, 19998,
       19998, 19998, 19998])

In [81]:
max_len = 0
for i in range(len(eval_sentences)):
    if (len(eval_sentences[i].split()) > max_len and len(eval_sentences[i].split()) < 29):
        max_len = len(eval_sentences[i].split())
        print('New max at ', i, ' : ', max_len, ' : ', eval_sentences[i])

New max at  0  :  21  :  he took my face in his hands and held my face where he wanted it as he kissed me senseless .
New max at  7  :  26  :  so that when we go meet your family , there is no doubt to anyone looking at us that you 're happy and confident . ''
New max at  16  :  28  :  `` logic does n't mesh with the fact that you 're a raging bitch , too , and you hate me , '' ty pointed out drolly .


In [78]:
len(eval_sentences)

10001